# 🌈 Quickstart

Want to get started right away? You've come to the right place!

In [ ]:
from chromatic import *

Let's create a blank simulated light curve, with some noise in it.

In [ ]:
r = SimulatedRainbow(dt=1 * u.minute, R=50)

Let's inject a transit into it.

In [ ]:
t = r.inject_transit(planet_radius=np.random.normal(0.1, 0.01, r.nwave))

Let's inject some noise into that transit.

In [ ]:
i = t.inject_noise(signal_to_noise=100)

Let's display it with color representing the brightness at each wavelength/time point.

In [ ]:
i.imshow();

Let's bin it, in wavelength and time.

In [ ]:
b = i.bin(dw=0.5 * u.micron, dt=20 * u.minute)

Let's do all of those actions in one (probably overly complicated) single command.

In [ ]:
(
    SimulatedRainbow(dt=1 * u.minute, R=50)
    .inject_transit(planet_radius=np.random.normal(0.1, 0.01, r.nwave))
    .inject_noise(signal_to_noise=100)
    .bin(dw=0.5 * u.micron, dt=20 * u.minute)
    .imshow()
);

Here you've seen some of the basic functionality of what `chromatic` can do. To figure out what other options are available for any particular `Rainbow` object, you can run the `.help()` method, like this: 

In [ ]:
r.help()

Please read on through the documentation to learn about reading a 🌈 from a real data file, doing actions on/with a 🌈, visualizing a 🌈 in different ways, and more! 